In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
from plotting import plot_ts

### Возьмем временной ряд

In [5]:
from load_dataset import Dataset

In [6]:
ds = Dataset('../data/dataset/')

In [7]:
ts = ds['daily-min-temperatures.csv']

### Cоздание инстанса

In [8]:
from model import TimeSeriesPredictor

в качестве обязательных параметров принимает частоту ряда в формате iso8601
и количество лагов для построения модели - granularity и num_lags

In [9]:
predictor = TimeSeriesPredictor('P1D', 365)

также predictor'y можно передать параметр model, чтобы использовать конкретную модель для прогнозирования, модель может быть любым sklearn-совместимым эстиматором

In [10]:
from sklearn.ensemble import GradientBoostingRegressor

predictor = TimeSeriesPredictor('P1D', 365, model=GradientBoostingRegressor)

model по умолчанию - LinearRegression

In [11]:
predictor = TimeSeriesPredictor('P1D', 365)
predictor.model

LinearRegression()

также для передачи дополнительной информации в модель вы можете использовать параметр mappers куда нужно передать ваши функции, принимающие timestamp и возвращающие для него значение вашего доп признака

In [12]:
def get_hour(timestamp):
    return timestamp.hour

external_features = {
    'hour': get_hour
}

predictor = TimeSeriesPredictor('P1D', 365, mappers=external_features)

Помимо основных параметровв модель также можно передать любое произвольно количество именованных аргументов **kwargs, которые будут переданы в конструктор `model`

In [13]:
from sklearn.ensemble import GradientBoostingRegressor

predictor = TimeSeriesPredictor('P1D', 365, model=GradientBoostingRegressor, max_depth=6, n_estimators=1000)

### Получение и установка параметров и функционал get/set params

классическое получение и задание аттрибутов

In [14]:
predictor.num_lags
predictor.model
predictor.model.learning_rate

0.1

In [15]:
predictor.model.learning_rate = 0.2

In [16]:
predictor.model.learning_rate

0.2

метод get_params получает словарь всех параметров модели включая sub-параметры модели, в таком случае они идут с префиксом
`model__`

In [17]:
predictor.get_params()

{'granularity': 'P1D',
 'mappers': {},
 'model__alpha': 0.9,
 'model__ccp_alpha': 0.0,
 'model__criterion': 'friedman_mse',
 'model__init': None,
 'model__learning_rate': 0.2,
 'model__loss': 'squared_error',
 'model__max_depth': 6,
 'model__max_features': None,
 'model__max_leaf_nodes': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__n_estimators': 1000,
 'model__n_iter_no_change': None,
 'model__random_state': None,
 'model__subsample': 1.0,
 'model__tol': 0.0001,
 'model__validation_fraction': 0.1,
 'model__verbose': 0,
 'model__warm_start': False,
 'model': GradientBoostingRegressor(learning_rate=0.2, max_depth=6, n_estimators=1000),
 'num_lags': 365}

метод set_params реализует обратную логику, принимая набор параметров для установки, включая sub-параметры модели, в таком случае их нужно указывать с префиксом `model__`

In [18]:
from sklearn.ensemble import RandomForestRegressor


params = {
    'num_lags': 730,
    'model': RandomForestRegressor(),  # обратите внимание, что в данном случае требуется создать инстанс модели!!
    'model__max_depth': 3,
    'model__n_estimators': 400,
}

predictor.set_params(**params)  # параметры необходимо распаковать

In [19]:
predictor.get_params()

{'granularity': 'P1D',
 'mappers': {},
 'model__bootstrap': True,
 'model__ccp_alpha': 0.0,
 'model__criterion': 'squared_error',
 'model__max_depth': 3,
 'model__max_features': 1.0,
 'model__max_leaf_nodes': None,
 'model__max_samples': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__n_estimators': 400,
 'model__n_jobs': None,
 'model__oob_score': False,
 'model__random_state': None,
 'model__verbose': 0,
 'model__warm_start': False,
 'model': RandomForestRegressor(max_depth=3, n_estimators=400),
 'num_lags': 730}

###  Обучение модели

в данном случае на вход подается временной ряд в формате pd.Series

In [20]:
split_idx = int(len(ts) * 0.7)

ts_train, ts_test = ts[:split_idx], ts[split_idx:]

In [ ]:
predictor.fit(ts_train)

### Получение out-of-sample прогноза

на вход подаем временной ряд от которого нужно сделать прогноз (длинной минимум num_lags) + горизонт прогнозирования

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
preds = predictor.predict_next(ts_train, n_steps=len(ts_test))

In [ ]:
plot_ts(ts_test, preds)

In [ ]:
mse(ts_test, preds)

### Получение in-sample (когда каждая точка нам известна) прогноза

в таком случае мы должны вторым аргументов передать реальный тест, такм образом этим методом мы смотрим как бы мы предсказали ts_test если бы мы предсказывали на одну точку вперед, т.е. нам всегда известна реальная предыдущая точка

In [ ]:
preds = predictor.predict_batch(ts_train, ts_test)

In [ ]:
plot_ts(ts_test, preds)

In [ ]:
mse(ts_test, preds)